# UDACITY : Designing Your First Workflow - Invoking Lambda Functions

In the last exercise, you created your own Lambda function. Without realizing it, you've already practiced invoking as well! Launching a test event is an example of synchronous invocation. In this exercise, you will continue working on the lambda function 'PreprocessLambda' from the previous exercise. However, you'll practice a different way to launch asynchronous invocation, and also practice the setup of an asynchronous invocation.
These are only two examples. Lambda is one of the most flexible offerings in AWS and can be utilized in a variety of applications. The same Lambda function can be (and often is) both invoked synchronously and asynchronously.

## Exercise: Synchronous invocation 

Synchronous invocations occur when a call is made to a Lambda function and a response is waited for. The example we're asking you to implement is a CLI invocation, but Lambda functions can also be placed behind Amazon's API Gateway for potential users to directly invoke a Lambda function. This, in turn, could be the interface that you expose to users to allow them to interact with other AWS resources. These types of invocations are great for "get" and "set" methods.

Your task is to synchronously invoke the Lambda function you implemented in the last exercise using the CLI. The following documentation may be useful to you: https://docs.aws.amazon.com/lambda/latest/dg/invocation-sync.html 

You will need to attach the LambdaFullAccess policy to the SageMaker execution role used for your notebook. Once done, it will take a few minutes for the policy to register.  

In [2]:
from sagemaker import get_execution_role
role = get_execution_role()
role

'arn:aws:iam::114256180253:role/service-role/AmazonSageMaker-ExecutionRole-20211127T235743'

In [13]:
%%bash
aws lambda invoke --function-name hello-lambda --payload '{ "key": "value" }' response.json

{
    "StatusCode": 200,
    "ExecutedVersion": "$LATEST"
}


## Exercise: Asynchronous invocation 

Asynchronous invocations occur when a service invokes lambda but does not wait for a response. The two most popular services that utilize asynchronous invocations are S3 (the storage we've been using) and SNS (Simple Notification Service.) We'll be setting up asynchronous invocations on an S3 bucket for our preprocessing function.

Your task is to setup a trigger for the Lambda function we've been working  whenever a file is uploaded to a specific folder in S3. You will need to do the following:

* Create a new s3 folder within an existing bucket. 
* Create a new lambda trigger for S3 by clicking '+Add trigger'. Specifying the bucket. Specify a prefix of the desired folder. Specify a suffix of ".zip" to ensure that recursive calls don't occur. 
* Modify the lambda handler in the previous exercise using the starter code so that it properly parses the event that's sent to it. 

To test, upload reviews_Patio_Lawn_and_Garden_5.json.zip in this directory to your S3 bucket. 
To see if the lambda function is triggered, you can go to the Monitor tab. 



## Lambda Handler Starter Code: Parsing S3 Upload Event. 

In [ ]:
# Todo: write a lambda_handler function here.
# The code to parse S3 event has provided to you, you only need to call the `preprocess` from the HelloBlazePreprocessLambda.py and return the status.
import json
import urllib

'''
{
    "Records": [
        {
            "eventVersion": "2.1",
            "eventSource": "aws:s3",
            "awsRegion": "us-east-1",
            "eventTime": "2021-12-15T02:33:38.448Z",
            "eventName": "ObjectCreated:Put",
            "userIdentity": {
                "principalId": "AWS:AROARVGRS6QOVSFEXGQFW:user1680226=11299473089"
            },
            "requestParameters": {
                "sourceIPAddress": "181.188.160.149"
            },
            "responseElements": {
                "x-amz-request-id": "7Q3BAY680CVSN9MH",
                "x-amz-id-2": "gvhPcJBLXJ1okqpTZWvyxCJW4uQPabavjo4lK0fkKUJr8kzvRC+8HQ3d3nv5d2B33d0UtArY+Et6XmMe2QXpdjfeOLA7GgJs2dAJ15lGpn0="
            },
            "s3": {
                "s3SchemaVersion": "1.0",
                "configurationId": "3961582c-de26-4fd8-a116-e3ac341bf357",
                "bucket": {
                    "name": "edgarin-mlend-c2",
                    "ownerIdentity": {
                        "principalId": "A1RJ3JL50VKM38"
                    },
                    "arn": "arn:aws:s3:::edgarin-mlend-c2"
                },
                "object": {
                    "key": "s3-events/reviews_Patio_Lawn_and_Garden_5.json.zip",
                    "size": 4981803,
                    "eTag": "a9679b8a406b7f35cbe79e3fa5fa63eb",
                    "sequencer": "0061B954025A1C81D6"
                }
            }
        }
    ]
}
'''

for r in event['Records']:
        bucket = r['s3']['bucket']['name']
        key = urllib.parse.unquote_plus(r['s3']['object']['key'], encoding='utf-8')
        uri = "/".join([bucket, key])